# Authentication

One thing you may have already noticed is that the way we've been using a few different techniques to authenticate ourselves when we're using Azure services -- we accessed our Azure Storage Account with a long key, but we accessed our Workspace by having Python open a browser window where we had to log in. So... what's the deal?

In short, there are two types of authentication in Azure (and many cloud platforms): keys (or secrets), and a managed access system called Azure Active Directory (or AD).

## Secret Keys

Keys (secrets) are long strings of numbers/letters/symbols that are used as passwords for various services. For example, when we accessed  accessed our storage by using a very long secret key. Keys are really easy to use initially (just copy and paste the string!), but for reasons we'll discuss below, they're actually much less secure and can lead to all sorts of problems. And so because of that, Azure also has...

## Azure Active Directory (Azure AD)

Azure Active Directory (AD) is a service provided by Azure for managing access to Azure resources and services. Basically, instead of managing secret keys yourself, you just tell Azure who you want to have access to resources, and people log-in through Azure to get access. It's exactly the same as the system that underlies the "Sign-in with Google" or "Sign-in with Facebook" buttons you see all over the web. 

## Why Do We Have Both / Which Should I Use?

For a long time, keys were the only way to manage access to computer systems, and they are still often times the easiest to use. But they also have a *lot* of security problems:


- **They aren't personalized:** Suppose that somehow your secret key gets out -- you inadvertently put it in a public github repo file (this is so common github has a system that scans for them and sends you warnings when you screw up), or maybe you aren't sure you've leaked a key, but you realize a resource you created might have had a security hole, so it *may* have been leaked. You can always cancel a secret key, but if you do so, it will immediately break *everything* your using that uses that key! If you have a key that only controls access to a small Storage Account only you use, that may not be a big problem, but what if dozens of people use a shared resource? You'd have to re-distibute the new key to all of them, and they'd have to update it in all the systems they use! So with keys, a small leak can result *huge* headaches.
- **Hard to cycle:** Similarly, its good security practice to cycle your keys, meaning that from time to time you change them (just like your password!). But with a hard-coded key, that causes problems because you may have many resources (or many team members) all using a specific key. As a result, changing that key may cause massive headaches for everyone. 
- **No scope:** With a key, people usually either have the key or they don't, making access binary. You (usually) can't create a key that only gives people read-only access, and a separate key that gives people edit access, etc, and you *definitely* can't chang people's permissions once they have a given key.

Azure AD solves all these problems. When someone signs in to a service with Azure AD, they're actually given a secret key behind the scenes. But here's the great part: that key is unique to that user, *and* it has an expiration date. That means that: 

- **Leaks are contained:** If someone accidentally gets a virus that steals their login info, or someone on your team gets fired, you can revoke *just that person's* access. 
- **Cycling is Automatic:** Keys provided by Azure AD automatically expire after a set period, requiring users to re-authenticate. You may not realize it, but you've probably seen this happening with other access management systems. You know how GMail makes you re-enter you password every thirty days? That's because the secret key it gave you last time you entered your password has expired so you need to login and get a new one. 
- **Scope:** The other great thing about Azure AD is that you can specify specific permissions for different users (read-only, read and edit, etc.). Moreover, you can update these permissions without breaking everyone's workflow! 

Of course, this *does* mean that you're moving all your trust from your key to people's Azure logins. But whereas keys are just secret strings of letters and numbers, Azure AD logins are much more secure -- it generally requires two-factor authentication, and (I *think*) it tracks login behavior for suspicious activity (e.g. if I try and login from Duke in the morning and Moscow at lunch, my guess it that Azure would ask for some extra verifications). 

So what service is best? In general, Azure AD is more secure and often easier to use, so you probably want to use it when you can, but there are still situations where you might want to use a key. For example...

## SSH Authentication 

While *most* of the time we have the option of easily accessing Azure services using Azure AD, one exception is when you want to ssh into a virtual machine. (Azure has something for this, [Key Vault](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/key-vault-setup), but not gonna lie: seems super complicated...)

`ssh`ing is when you connect to a virtual machine from the command line (e.g. your command line session becomes a command line session on the virtual machine) using the **S**ecure **SH**ell protocol. This is great both for working on a virtual machine, and also moving files back and forth (as both [scp](https://www.garron.me/en/articles/scp.html) and [rsync](https://www.digitalocean.com/community/tutorials/how-to-use-rsync-to-sync-local-and-remote-directories-on-a-vps) --- the easiest tools for transfering data between computers from the command line --- run on ssh.) If you don't know much about ssh, here's a [probably-a-little-too-in-depth-but-good tutorial](`https://www.digitalocean.com/community/tutorials/ssh-essentials-working-with-ssh-servers-clients-and-keys`).

There are a couple ways to connect two computers using ssh. The first is to just connect normally (type `ssh <username>@<hostcomputeraddress>`) then answer the password prompts. The problem with this strategy, however, is that Azure doesn't support password prompts because user-created passwords tend to be wildly insecure. 

So the second option to use an RSA keypair, which is where you generate a pair of matching keys on your own computer (they get stored in `~/.ssh`), then give one to the VM you want to connect with at setup. From that point forward, whenever you type `ssh <username>@<hostcomputeraddress>` (or use `scp` to copy files), your computer will automatically check these codes and log you in automatically. 

You can find a tutorial on setting up ssh keypairs on [Mac or Linux here](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/mac-create-ssh-keys), and on [Windows here](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/ssh-from-windows). 


## Conclusion

So what authentication system should you use?

If your system is just for you, and your data's not super sensitive, using keys is probably fine. But Azure AD is better, safer, and often just as easy to use. And if you're working with a *team*, I dare say it's a must.